In [2]:
import mmcv
import numpy as np
from pycocotools.coco import COCO
#from mmdet.core.evaluation.eval_mIOU import evaluate_mIOU


gt_fp = 'data/dataset/val.json'
submit_fp = '/home/arias/Datasets/jinnan2/submit/val/full_val_04-10_21:43_CascadeRCNN_ResNet_normal_may_not_mixup_mIOU_0.7325.json'
gt_coco = COCO(gt_fp)
submit_js = mmcv.load(submit_fp)

In [4]:
for thresh in np.arange(0.1, 1.0, 0.05):
    IOU, iou_list = evaluate_mIOU(gt_coco, submit_js, score_thresh=thresh, return_iou=True)
    msg = f'Thresh: {thresh:.2f}  mIOU: {IOU:.4f} '
    for i, iou in enumerate(iou_list):
        msg += f'{i+1}: {iou:.4f}  '
    print(msg)

Thresh: 0.10  mIOU: 0.6970 1: 0.8545  2: 0.6072  3: 0.6844  4: 0.7589  5: 0.5798  
Thresh: 0.15  mIOU: 0.7110 1: 0.8585  2: 0.6224  3: 0.7032  4: 0.7742  5: 0.5964  
Thresh: 0.20  mIOU: 0.7172 1: 0.8585  2: 0.6309  3: 0.7105  4: 0.7822  5: 0.6038  
Thresh: 0.25  mIOU: 0.7233 1: 0.8560  2: 0.6371  3: 0.7210  4: 0.7923  5: 0.6100  
Thresh: 0.30  mIOU: 0.7261 1: 0.8580  2: 0.6434  3: 0.7173  4: 0.7959  5: 0.6161  
Thresh: 0.35  mIOU: 0.7284 1: 0.8620  2: 0.6424  3: 0.7235  4: 0.7962  5: 0.6180  
Thresh: 0.40  mIOU: 0.7293 1: 0.8620  2: 0.6440  3: 0.7213  4: 0.7991  5: 0.6202  
Thresh: 0.45  mIOU: 0.7317 1: 0.8620  2: 0.6440  3: 0.7237  4: 0.8042  5: 0.6248  
Thresh: 0.50  mIOU: 0.7325 1: 0.8620  2: 0.6402  3: 0.7307  4: 0.8029  5: 0.6266  
Thresh: 0.55  mIOU: 0.7286 1: 0.8561  2: 0.6345  3: 0.7307  4: 0.7985  5: 0.6233  
Thresh: 0.60  mIOU: 0.7255 1: 0.8510  2: 0.6325  3: 0.7278  4: 0.7940  5: 0.6222  
Thresh: 0.65  mIOU: 0.7231 1: 0.8510  2: 0.6252  3: 0.7255  4: 0.7936  5: 0.6205  
Thre

In [1]:
import mmcv
import numpy as np
from config import cfg
from pycocotools import mask as mask_utils
from mmdet.core.evaluation.miou import mIOU


def get_gt_masks(anns, coco, img_h, img_w):
    num_classes = len(coco.getCatIds())
    masks = np.zeros(shape=(num_classes, img_h, img_w), dtype='uint8')
    for ann in anns:
        category_id = ann['category_id']
        mask = coco.annToMask(ann)
        masks[category_id - 1] += mask
    masks = np.clip(masks, a_min=0, a_max=1)
    return masks


def get_pred_masks(anns, coco, img_h, img_w):
    num_classes = len(coco.getCatIds())
    masks = np.zeros(shape=(num_classes, img_h, img_w), dtype='uint8')
    for ann in anns:
        category_id = ann['category_id']
        if not 'mask' in ann:
            ann['mask'] =  mask_utils.decode(ann['segmentation'])
        mask = ann['mask']
        masks[category_id - 1] += mask
    masks = np.clip(masks, a_min=0, a_max=1)
    return masks


def evaluate_mIOU(gt_coco, result_json, score_thresh=None, return_iou=False):
    """
    :param score_thresh: None means use cfg.test_cfg.rcnn.mask_nms_thresh, -1 means disable score thresh
    """
    if isinstance(result_json, str):
        result_json = mmcv.load(result_json)

    if score_thresh is None:
        score_thresh = cfg.test_cfg.rcnn.mask_nms_thresh
    valid_result_json = [ann for ann in result_json if ann['score']>score_thresh]
    evaluator = mIOU(len(gt_coco.getCatIds()))
    img_ids = gt_coco.getImgIds()
    img_id_to_anns_dict = _build_img_id_to_anns_dict(valid_result_json)
    for img_id in img_ids:
        gt_img_info = gt_coco.loadImgs(img_id)[0]
        img_h = gt_img_info['height']
        img_w = gt_img_info['width']
        gt_ann_ids = gt_coco.getAnnIds(img_id)
        gt_anns = gt_coco.loadAnns(gt_ann_ids)
        gt_masks = get_gt_masks(gt_anns, gt_coco, img_h=img_h, img_w=img_w)

        pred_anns = img_id_to_anns_dict.get(img_id, [])
        pred_masks = get_pred_masks(pred_anns, gt_coco, img_h=img_h, img_w=img_w)

        evaluator.add_batch(gt_masks, pred_masks)

    return evaluator.calc_miou(return_iou=return_iou)


def _build_img_id_to_anns_dict(result_json):
    d = dict()
    for ann in result_json:
        img_id = ann['image_id']
        d.setdefault(img_id, [])
        d[img_id].append(ann)
    return d